# **Extracting Semantic Model Memory Size from the Fabric Capacity Metrics App for Multiple Capacities**

# Check out this [blog post](https://bits2bi.com/2025/03/15/extracting-semantic-model-size-from-the-fabric-capacity-metrics-app) for details about this notebook

⚠️ Note
The DAX query to extract the semantic model size mentioned in this Notebook was written and tested against the Fabric Capacity Metrics App version 1031 (updated on September 11, 2025). It may not work with older or newer versions, as the semantic model structure and object names are subject to change.

In [ ]:
# Install the latest version of semantic link

## ........ UNCOMMENT THE FOLLOWING LINE IF YOU'RE RUNNING SPARK 3.3 OR BELOW OR TO UPDATE THE SEMANTIC LINK TO THE LATEST VERSION ........ ##

#%pip install -U semantic-link

In [ ]:
# Import the Semantic Link Python library
import sempy.fabric as fabric

# Import other libraries and modules
from pyspark.sql.functions import lit, current_timestamp, to_date, col

import pandas as pd

In [ ]:
# Declare and assign variables

## ........ PROVIDE THE NAME OR ID OF YOUR FABRIC CAPACTIY METRICS WORKSPACE AND FABRIC CAPACTIY METRICS SEMANTIC MODEL ........ ##

workspace_Name = "<WORKSPACE_NAME_OR_ID>"
dataset_Name = "<SEMANTIC_MODEL_NAME_OR_ID>"

### Are you a capacity admin who wants to extract semantic model size information from a Fabric Capacity Metric App you've installed?
##### - **Yes:** Unfreeze the FROZEN CELL #1
##### - **No:** Unfreeze the FROZEN CELL #2

**<mark>_These capacities should be accessible from the Fabric Capacity Metric semantic model you specified in the previous cell. This Notebook will not work as-is if the capacities are inaccessible from a single Capacity Metrics App._</mark>**

In [ ]:
## ........ FROZEN CELL #1 ........ ##

## Get the list of capacities you have access to. These capacities should be accessible from the Fabric Capacity Metrics semantic model you specified in the previous cell.
## The list_capacities() returns a list of capacities to which you have Admin and Contributor rights.

## ........ IF YOU'VE CONTRIBUTOR RIGHTS TO A CAPACITY, THEN THE CAPACITY IS NOT ACCESSIBLE FROM THE FABRIC CAPACITY METRIC APP THAT YOU'VE INSTALLED ........ ##
## ........ IN THIS CASE, YOU HAVE TO PROVIDE THE LIST OF CAPACITIES MANUALLY, PROCEED TO FROZEN CELL 2 ........ ##

# https://learn.microsoft.com/en-us/python/api/semantic-link-sempy/sempy.fabric?view=semantic-link-python#sempy-fabric-list-capacities
capacities = fabric.list_capacities()

# Keep only the Id column
capacities_df = capacities[['Id']]

display(capacities_df)

In [ ]:
## ........ FROZEN CELL #2 ........ ##

## ........ PROVIDE THE LIST OF CAPACITIES TO WHICH YOU'VE RECEIVED ACCESS TO THEIR FABRIC CAPACITY METRICS APP. THESE CAPACITIES SHOULD BE ACCESSIBLE FROM THE FABRIC CAPACITY METRIC SEMANTIC MODEL YOU SPECIFIED IN THE PREVIOUS CELL ........ ##

capacities = {
    'Id': ["CAPACITY_ID1","CAPACITY_ID2","CAPACITY_ID3"]
}
capacities_df = pd.DataFrame(capacities)

display(capacities_df)

In [ ]:
# Define the function to call fabric.evaluate_dax and return the result as a DataFrame
def evaluate_dax_and_return_df(capacity_Id):

    dax_query = f"""
    // DAX query to return the memory usage statistics for the previous day
    // for all the semantic models hosted on a capacity
    DEFINE
    // Provide a capacity ID for the MPARAMETER
        MPARAMETER 'CapacitiesList' = "{capacity_Id}"
    // Filter the items for Datasets as we only want the memory information for semantic models
        VAR __DS0FilterTable =
            TREATAS ( {{ "Dataset" }}, 'Items'[Item kind] )
    // Filter for yesterday as today's statistics are not yet final and may change during the day
    // Always extract the data for the completed days
        VAR __DS0FilterTable2 =
            TREATAS ( {{ UTCTODAY () - 1 }}, 'Dates'[Day] )
    // Extract Minimum, Maximum, and Median memory sizes
        VAR __DS0Core =
            SUMMARIZECOLUMNS (
                'Dates'[Day],
                Items[Item Id],
                Items[Item name],
                __DS0FilterTable,
                __DS0FilterTable2,
                "MinimumMemoryInGB", ROUND ( MIN ( 'Max Memory By Item And Hour'[Item size (GB)] ), 4 ),
                "MaximumMemoryInGB", ROUND ( MAX ( 'Max Memory By Item And Hour'[Item size (GB)] ), 4 ),
                "MedianMemoryInGB", ROUND ( MEDIAN ( 'Max Memory By Item And Hour'[Item size (GB)] ), 4 )
            )

    EVALUATE
    __DS0Core
"""

    # Execute DAX query on the semantic model
    # https://learn.microsoft.com/en-us/python/api/semantic-link-sempy/sempy.fabric?view=semantic-link-python#sempy-fabric-evaluate-dax
    dax_result = fabric.evaluate_dax(
        workspace=workspace_Name,
        dataset=dataset_Name,
        dax_string=dax_query
    )
    # Convert the result to a pandas DataFrame
    result_df = pd.DataFrame(dax_result)
    return result_df

# Apply the function to each row and collect the results into a list
results = capacities_df['Id'].apply(lambda x: evaluate_dax_and_return_df(x)).tolist()

# Combine the list of DataFrames into a single DataFrame, ignoring empty DataFrames
combined_dax_result_df = pd.concat([df for df in results if not df.empty], ignore_index=True)

# Show the result
display(combined_dax_result_df)

In [ ]:
# Convert the DAX result to spark dataframe
dax_result_df = spark.createDataFrame(combined_dax_result_df)

# Add the current time as data extraction date
dax_result_df = dax_result_df.withColumn("ExtractionDate", lit(current_timestamp()))

dax_result_df.printSchema()

In [ ]:
# Rename the columns for clarity
dax_result_cleaned_df = dax_result_df.withColumnRenamed("Dates[Day]", "Date") \
               .withColumn("Date", to_date(col("Date")))\
               .withColumnRenamed("Items[Item Id]", "SemanticModelID") \
               .withColumnRenamed("Items[Item Name]", "SemanticModelName") \
               .withColumnRenamed("[MinimumMemoryInGB]", "MinimumMemoryInGB") \
               .withColumnRenamed("[MaximumMemoryInGB]", "MaximumMemoryInGB") \
               .withColumnRenamed("[MedianMemoryInGB]", "MedianMemoryInGB")

# Total rows
print(f"Total Rows: {dax_result_cleaned_df.count()}")

dax_result_cleaned_df.printSchema()

# Show the data
display(dax_result_cleaned_df)

In [ ]:
# Append the data to the lakehouse delta table
dax_result_cleaned_df.write.format("delta").mode("append").saveAsTable("semantic_model_size_metrics")